In [28]:
import os
import random
import math

import numpy as np
import haven.db as db 

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [34]:
TEST = True

table = 'movement_model_raw_features_m1_a1' if not TEST else 'movement_model_raw_features_m1_a1_test'
data = read_data_w_cache(
    f'select * from {table}'
)
data = data[data['_selected'].isin([True, False])]
data.head()

,_decision,_choice,_selected,_individual,normed_distance,origin_h3_index,next_h3_index,h3_index,time,tag_key
0,1,1,False,142194,0.408506,8422d0bffffffff,8422d03ffffffff,8422d01ffffffff,2015-11-21,142194
1,1,2,True,142194,0.424598,8422d0bffffffff,8422d03ffffffff,8422d03ffffffff,2015-11-21,142194
2,1,3,False,142194,0.814645,8422d0bffffffff,8422d03ffffffff,8422d05ffffffff,2015-11-21,142194
3,1,4,False,142194,0.702011,8422d0bffffffff,8422d03ffffffff,8422d07ffffffff,2015-11-21,142194
4,1,5,False,142194,0.446883,8422d0bffffffff,8422d03ffffffff,8422d09ffffffff,2015-11-21,142194


In [35]:
df = data.merge(
    data[['_individual']].drop_duplicates().reset_index(drop=True).reset_index(),
    on='_individual'
)
del df['_individual']
df = df.rename({'index': '_individual'}, axis=1)
df.head()

,_decision,_choice,_selected,normed_distance,origin_h3_index,next_h3_index,h3_index,time,tag_key,_individual
0,1,1,False,0.408506,8422d0bffffffff,8422d03ffffffff,8422d01ffffffff,2015-11-21,142194,0
1,1,2,True,0.424598,8422d0bffffffff,8422d03ffffffff,8422d03ffffffff,2015-11-21,142194,0
2,1,3,False,0.814645,8422d0bffffffff,8422d03ffffffff,8422d05ffffffff,2015-11-21,142194,0
3,1,4,False,0.702011,8422d0bffffffff,8422d03ffffffff,8422d07ffffffff,2015-11-21,142194,0
4,1,5,False,0.446883,8422d0bffffffff,8422d03ffffffff,8422d09ffffffff,2015-11-21,142194,0


In [36]:
individuals = list(df['_individual'].unique())
test = random.sample(individuals, math.ceil(len(individuals) * 0.25))
if not TEST:
    df['_train'] = ~df['_individual'].isin(test)
else:
    df['_train'] = False
df.head()

,_decision,_choice,_selected,normed_distance,origin_h3_index,next_h3_index,h3_index,time,tag_key,_individual,_train
0,1,1,False,0.408506,8422d0bffffffff,8422d03ffffffff,8422d01ffffffff,2015-11-21,142194,0,False
1,1,2,True,0.424598,8422d0bffffffff,8422d03ffffffff,8422d03ffffffff,2015-11-21,142194,0,False
2,1,3,False,0.814645,8422d0bffffffff,8422d03ffffffff,8422d05ffffffff,2015-11-21,142194,0,False
3,1,4,False,0.702011,8422d0bffffffff,8422d03ffffffff,8422d07ffffffff,2015-11-21,142194,0,False
4,1,5,False,0.446883,8422d0bffffffff,8422d03ffffffff,8422d09ffffffff,2015-11-21,142194,0,False


In [37]:
df.groupby('_train').size()

_train
False    27178
dtype: int64

In [38]:
table = 'movement_model_features_m1_a1' if not TEST else 'movement_model_features_m1_a1_test'

db.write_data(
    df, table, ['tag_key']
)